# DePlan Complete Workflow Example

This notebook demonstrates the complete workflow of the DePlan project:
1. **Task Loading**: Natural language task descriptions example
2. **PDDL Generation**: Translating natural language to PDDL problem files using LLM
3. **Planning Solving**: Solving PDDL problems using the Fast-Downward solver
4. **Result Evaluation**: Analyzing planning results and performance metrics

## Example: Blocksworld Domain

We will use the first task from the Blocksworld domain as an example.


In [ ]:
import asyncio
import sys
from pathlib import Path

# Add project root to path
project_root = Path.cwd()
sys.path.insert(0, str(project_root))

from agents.deplan.agent import DePlanAgent
from envs.pddl.env import PDDLEnv

print("✅ Imports completed")


## Step 1: Initialize Environment and Agent

First, we need to:
- Initialize the PDDL environment (load domain files and tasks)
- Initialize the LLM Agent (configure LLM client)


In [ ]:
# Configuration parameters
config = {
    "profile": "deepseek",  # LLM configuration name (needs to be configured in configs/profiles.yaml)
    "domain_name": "blocksworld",
    "task_id": 0,  # Use the first task
    "use_context": False,  # Whether to use in-context learning
}

# Initialize environment
print("📦 Initializing PDDL environment...")
env = PDDLEnv(domain_name=config["domain_name"])

# Initialize Agent
print("🤖 Initializing DePlan Agent...")
agent = DePlanAgent(use_context=config["use_context"])

print("✅ Initialization completed")


## Step 2: Load Task

The environment will load the specified task, including:
- Natural language task description (.nl file)
- Standard PDDL problem file (.pddl file, for comparison)
- Domain description (domain.nl and domain.pddl)


In [ ]:
# Reset environment to load task
print("🔄 Loading task...")
init_info = env.reset(config, id=str(config["task_id"]))
agent.reset(config, init_info)

# Get task information
task_nl = init_info["observations"][0]
domain_nl = init_info["domain_nl"]
domain_pddl = init_info["domain_pddl"]

print("=" * 60)
print("📝 Natural Language Task Description:")
print("=" * 60)
print(task_nl)
print("\n" + "=" * 60)
print("📚 Domain Description (Natural Language):")
print("=" * 60)
print(domain_nl[:500] + "..." if len(domain_nl) > 500 else domain_nl)


In [ ]:
# Display standard PDDL problem file (for comparison)
print("=" * 60)
print("📋 Standard PDDL Problem File (Reference):")
print("=" * 60)
print(env.current_task_pddl)


## Step 3: LLM PDDL Generation

The Agent uses LLM to translate natural language task descriptions into PDDL problem files.


## Step 3.5: Display Complete LLM Message

Display the complete message sent to the LLM, including the prompt and system message (if any).


In [ ]:
# Build the prompt that was sent to LLM
task_nl = init_info["observations"][0]
if agent.use_context and agent.context:
    prompt = agent._build_llm_ic_pddl_prompt(task_nl)
else:
    prompt = agent._build_llm_pddl_prompt(task_nl)

# Build complete messages (including system prompt if any)
messages = agent.llm_client._build_messages(prompt, system_prompt=None)

print("=" * 60)
print("📨 Complete LLM Message:")
print("=" * 60)
print("\n")
for i, msg in enumerate(messages, 1):
    print(f"Message {i}:")
    print(f"  Role: {msg['role']}")
    print(f"  Content:")
    print("-" * 60)
    print(msg['content'])
    print("-" * 60)
    print("\n")
print("=" * 60)


In [ ]:
# Agent generates PDDL
print("🧠 LLM is generating PDDL problem file...")
observations = init_info["observations"]
actions = await agent.act(observations)

generated_pddl = actions[0]

print("=" * 60)
print("✅ Generated PDDL Problem File:")
print("=" * 60)
print(generated_pddl)
print("\n" + "=" * 60)
print(f"📊 Statistics:")
print(f"   - PDDL length: {len(generated_pddl)} characters")
print(f"   - LLM query count: {agent.num_queries}")
print(f"   - LLM cost: ${agent.total_cost:.4f}")
print("=" * 60)


## Step 4: Planning Solving

The environment uses the Fast-Downward solver to solve the generated PDDL problem.


In [ ]:
# Environment solves PDDL
print("⚙️  Calling Fast-Downward solver...")
observations = await env.run(actions)

print("=" * 60)
print("📤 Solver Output:")
print("=" * 60)
print(observations)


## Step 5: Evaluate Results

Check whether planning succeeded and analyze performance metrics.


In [ ]:
# Check results
success = env.is_success()
env_report = env.report()
agent_report = agent.report()

print("=" * 60)
print("📊 Evaluation Results")
print("=" * 60)
print(f"✅ Planning status: {'Success' if success else 'Failed'}")
print(f"\n📈 Environment Metrics:")
print(f"   - Planning cost: {env_report['cost']}")
print(f"   - Planning time: {env_report['time']:.2f} seconds")
print(f"   - Number of steps: {env_report['steps']}")
print(f"   - Task type: {env_report['task_type']}")
print(f"   - Task ID: {env_report['task_id']}")

print(f"\n🤖 Agent Metrics:")
print(f"   - LLM query count: {agent_report['llm_queries']}")
print(f"   - LLM total cost: ${agent_report['llm_cost']:.4f}")

if success:
    print(f"\n📋 Generated Planning Solution:")
    print("=" * 60)
    plan_lines = env.plan.split('\n') if env.plan else []
    for i, line in enumerate(plan_lines[:20], 1):  # Display first 20 lines
        print(f"{i:2d}. {line}")
    if len(plan_lines) > 20:
        print(f"... (total {len(plan_lines)} actions)")
    print("=" * 60)
else:
    print(f"\n❌ Planning failed, possible reasons:")
    print("   - PDDL syntax error")
    print("   - Problem is unsolvable")
    print("   - Solver timeout")
    print("   - Domain and problem mismatch")
print("=" * 60)


## Step 6: Comparative Analysis

Compare the differences between the generated PDDL and the standard PDDL.


In [ ]:
# Compare generated PDDL and standard PDDL
print("=" * 60)
print("🔍 PDDL Comparative Analysis")
print("=" * 60)

standard_pddl = env.current_task_pddl.strip()
generated_pddl_clean = generated_pddl.strip()

print("\n📋 Standard PDDL (Reference):")
print("-" * 60)
print(standard_pddl)

print("\n\n🤖 Generated PDDL:")
print("-" * 60)
print(generated_pddl_clean)

# Simple similarity check
if standard_pddl == generated_pddl_clean:
    print("\n✅ Generated PDDL is identical to standard PDDL!")
else:
    print("\n⚠️  Generated PDDL differs from standard PDDL")
    print(f"   Standard length: {len(standard_pddl)} characters")
    print(f"   Generated length: {len(generated_pddl_clean)} characters")
    
    # Check key elements
    key_elements = ["define", "problem", "domain", "objects", "init", "goal"]
    print("\n🔑 Key Element Check:")
    for element in key_elements:
        in_standard = element in standard_pddl.lower()
        in_generated = element in generated_pddl_clean.lower()
        status = "✅" if (in_standard == in_generated) else "❌"
        print(f"   {status} '{element}': standard={in_standard}, generated={in_generated}")

print("=" * 60)


| Category                  | Challenge                   | Core Symptom                       | Illustrative Example                                                                      |
| :------------------------ | :-------------------------- | :--------------------------------- | :---------------------------------------------------------------------------------------- |
| **Syntax Fragility**      | Formal-language errors      | Planner parsing failure            | Missing parentheses or undefined predicates — e.g., `(:action pick :precond (holding ?x)` |
| **Semantic Gap**          | Action/predicate mismatch   | Inconsistent preconditions/effects | `(open fridge)` without defining the object type or effect `(not (closed fridge))`        |
| **Feedback Sparsity**     | Planner only returns *fail* | No informative feedback signal     | “Plan-Not-Found” without showing which action failed                                      |                  |
